In [1]:
import pandas as pd

In [2]:
january = pd.read_parquet('data/january')

In [3]:
january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [4]:
january.shape

(2463931, 19)

Q1. Read the data for January. How many columns are there?
Answer = 19

In [5]:
january.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [6]:
january['tpep_pickup_datetime'] = pd.to_datetime(january['tpep_pickup_datetime'])
january['tpep_dropoff_datetime'] = pd.to_datetime(january['tpep_dropoff_datetime'])

In [7]:
january['duration'] = (january['tpep_dropoff_datetime'] - january['tpep_pickup_datetime']).astype('timedelta64[m]')

Q2. What's the standard deviation of the trips duration in January?
Answer = 46.445614433207076

In [8]:
january['duration'].std()

46.445614433207076

In [9]:
january_new = january[(january['duration'] <= 60 ) & (january['duration']) >= 1]

Q3. What fraction of the records left after you dropped the outliers?
Answer = %98

In [11]:
q3 = january_new.shape[0] / january.shape[0] 
q3 *= 100
q3

98.40689532296156

In [12]:
from sklearn.preprocessing import OneHotEncoder

In [13]:
encoder = OneHotEncoder(sparse_output = True)

In [28]:
encoder_new = OneHotEncoder(sparse_output = True, handle_unknown='ignore')

In [29]:
encoder_new.fit(january_new[['PULocationID','DOLocationID']])
feature_matrix_new = encoder_new.transform(january_new[['PULocationID','DOLocationID']])

In [14]:
encoder.fit(january_new[['PULocationID','DOLocationID']])
feature_matrix = encoder.transform(january_new[['PULocationID','DOLocationID']])

Q4. What's the dimensionality of this matrix (number of columns)?
Answer = 515

In [30]:
feature_matrix_new.shape

(2424678, 515)

In [15]:
feature_matrix.shape

(2424678, 515)

In [16]:
X = feature_matrix
y = january_new['duration']

In [17]:
from sklearn.linear_model import LinearRegression

In [18]:
model = LinearRegression(n_jobs = -1)
model.fit(X,y)

LinearRegression(n_jobs=-1)

In [21]:
from sklearn.metrics import mean_squared_error

Q5. Training a model? Answer = 7.4

In [33]:
february =  pd.read_parquet('data/february')
february['tpep_pickup_datetime'] = pd.to_datetime(february['tpep_pickup_datetime'])
february['tpep_dropoff_datetime'] = pd.to_datetime(february['tpep_dropoff_datetime'])
february['duration'] = (february['tpep_dropoff_datetime'] - february['tpep_pickup_datetime']).astype('timedelta64[m]')
february_new = february[(february['duration'] <= 60 ) & (february['duration']) >= 1]
X_val_ = encoder_new.transform(february_new[['PULocationID','DOLocationID']])
y_val = february_new['duration']

In [35]:
model.score(X_val_,y_val)

0.3459925237579222

Q6. Evaluating the model
Answer = 7.9

In [38]:
y_pred = model.predict(X_val_)
mean_squared_error(y_val, y_pred, squared=False)

7.900543377568379